In [ ]:
%cd /Users/robertford/Repos/fade
!source .venv/bin/activate
%uv pip install -e .
# %uv pip install matplotlib scikit-image notebook numpy pandas pillow scipy tqdm ipykernel jupyter paddlepaddle


In [ ]:
# Test cell to verify installations
import sys
print(f"Python version: {sys.version}")

# Test core dependencies
try:
    import matplotlib
    print(f"matplotlib version: {matplotlib.__version__}")
except ImportError as e:
    print(f"matplotlib import error: {e}")

try:
    import numpy
    print(f"numpy version: {numpy.__version__}")
except ImportError as e:
    print(f"numpy import error: {e}")

try:
    import pandas
    print(f"pandas version: {pandas.__version__}")
except ImportError as e:
    print(f"pandas import error: {e}")

try:
    import PIL
    print(f"Pillow version: {PIL.__version__}")
except ImportError as e:
    print(f"Pillow import error: {e}")

try:
    import skimage
    print(f"scikit-image version: {skimage.__version__}")
except ImportError as e:
    print(f"scikit-image import error: {e}")

try:
    import scipy
    print(f"scipy version: {scipy.__version__}")
except ImportError as e:
    print(f"scipy import error: {e}")

try:
    import paddle
    print(f"paddle version: {paddle.__version__}")
except ImportError as e:
    print(f"paddle import error: {e}")

# Test FADE package
try:
    from fade.pipeline import PipelineState
    print("FADE package imported successfully")
except ImportError as e:
    print(f"FADE package import error: {e}")

In [10]:
# Test the pipeline with a sample document
from fade.pipeline import (
    PipelineState,
    setup_working_directory,
    extract_document_pages,
    detect_entities,
    classify_entities,
    report_unclassified_entities,
    process_entities,
    log_process
)

import os

# Get absolute path to the PDF using the correct base path
base_path = "/Users/robertford/Repos/fade"  # Your project root
pdf_path = os.path.join(base_path, "data", "f1040.pdf")
print(f"PDF path: {pdf_path}")
print(f"File exists: {os.path.exists(pdf_path)}")

# Initialize a test state with absolute path
initial_state = PipelineState(
    document_id=pdf_path,  # Using absolute path
    working_dir="",
    images=[],
    entities={},
    unclassified_entities={},
    logs=[],
    error=None
)


PDF path: /Users/robertford/Repos/fade/data/510k_eSTAR.pdf
File exists: False


In [ ]:
# Test pipeline setup
print("\nTesting pipeline setup...")
state = setup_working_directory(initial_state)
print("\nSetup Working Directory Logs:")
for log in state.logs:
    print(f"- {log['message']}")
if state.error:
    print(f"Error: {state.error}")

In [ ]:
# Test document page extraction
print("\nTesting document page extraction...")
state = extract_document_pages(state)
print("\nExtract Document Pages Logs:")
for log in state.logs:
    print(f"- {log['message']}")
print(f"\nNumber of images extracted: {len(state.images)}")
if state.error:
    print(f"Error: {state.error}")

In [ ]:
# If we have images, proceed with entity detection
if len(state.images) > 0:
    print("\nTesting entity detection...")
    state = detect_entities(state)
    print("\nDetect Entities Logs:")
    for log in state.logs:
        print(f"- {log['message']}")
    print(f"\nNumber of entities detected: {len(state.entities)}")
    if state.error:
        print(f"Error: {state.error}")
else:
    print("\nSkipping entity detection since no images were extracted.")

In [ ]:
# Test entity detection
print("\nTesting entity detection...")
state = detect_entities(state)
print("\nDetect Entities Logs:")
for log in state.logs:
    print(f"- {log['message']}")
print(f"\nNumber of entities detected: {len(state.entities)}")
